- PassengerId - A unique Id for each passenger. Each Id takes the form gggg_pp where gggg indicates a group the passenger is travelling with and pp is their number within the group. People in a group are often family members, but not always.
- HomePlanet - The planet the passenger departed from, typically their planet of permanent residence.
- CryoSleep - Indicates whether the passenger elected to be put into suspended animation for the duration of the voyage. Passengers in cryosleep are confined to their cabins.
- Cabin - The cabin number where the passenger is staying. Takes the form deck/num/side, where side can be either P for Port or S for Starboard.
- Destination - The planet the passenger will be debarking to.
- Age - The age of the passenger.
- VIP - Whether the passenger has paid for special VIP service during the voyage.
- RoomService, FoodCourt, ShoppingMall, Spa, VRDeck - Amount the passenger has billed at each of the Spaceship Titanic's many luxury amenities.
- Name - The first and last names of the passenger.
- Transported - Whether the passenger was transported to another dimension. This is the target, the column you are trying to predict.

Plan:
1) CryoSleep, VIP, HomePlanet, Destination to labels -> 0.72
2) delete `Name`
3) PassengerId split into group & pp
4) Cabin split into deck & num  & side
5) NaNs

In [23]:
# import libraries
import pandas as pd

from sklearn.compose import make_column_transformer, make_column_selector, ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import Pipeline

from sklearn.metrics import accuracy_score

RANDOM_STATE = 654321

In [37]:
df = pd.read_csv('data/train.csv')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB
None


In [38]:
not_name = ['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

In [46]:
pipeline = Pipeline([
#    ('not_name', make_column_transformer(make_column_selector(pattern='^(Name)'))),
#    ('not_name', ColumnTransformer([('selector', 'passthrough', not_name)], remainder='drop')),
    ("name", ColumnTransformer([("selector", "passthrough", not_name)], remainder="drop")),
    ('ohe', make_column_transformer((OneHotEncoder(), (['CryoSleep', 'VIP', 'HomePlanet', 'Destination'])))),
    ('rfc', RandomForestClassifier(random_state=RANDOM_STATE))
])

In [47]:
y_train = df.Transported
X_train = df.drop(columns=['Transported'])

In [48]:
pipeline.fit(X_train, y_train)

ValueError: Specifying the columns using strings is only supported for pandas DataFrames

In [8]:
pred_train = pipeline.predict(X_train)

In [9]:
accuracy_score(y_train, pred_train)

0.7201029367241901

In [10]:
X_test = pd.read_csv('data/test.csv')


In [11]:
pred_test = pipeline.predict(X_test)

ValueError: Found unknown categories [nan] in column 0 during transform